In [1]:
# pip install tda-api

# from tda import auth, client
# import json

# import config_tda
# from tda import auth, client
# import json


# try:
#     c = auth.client_from_token_file(config_tda.token_path, config_tda.api_key)
# except FileNotFoundError:
#     from selenium import webdriver
#     with webdriver.Chrome() as driver:
#         c = auth.client_from_login_flow(
#             driver, api_key, redirect_uri, token_path)

# r = c.get_price_history('AAPL',
#         period_type=client.Client.PriceHistory.PeriodType.YEAR,
#         period=client.Client.PriceHistory.Period.TWENTY_YEARS,
#         frequency_type=client.Client.PriceHistory.FrequencyType.DAILY,
#         frequency=client.Client.PriceHistory.Frequency.DAILY)
# assert r.status_code == 200, r.raise_for_status()
# print(json.dumps(r.json(), indent=4))
# try:
#     c = auth.client_from_token_file(config_tda.token_path, config_tda.api_key)
# except FileNotFoundError:
#     from selenium import webdriver
#     with webdriver.Chrome() as driver:
#         c = auth.client_from_login_flow(
#             driver, api_key, redirect_uri, token_path)

# r = c.get_account(client.Client.Account.ORDERS)
# assert r.status_code == 200, r.raise_for_status()
# print(json.dumps(r.json(), indent=4))

In [2]:
import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
from dotenv import load_dotenv

from web3 import Web3
from pathlib import Path
import json
load_dotenv()

True

In [3]:

# Set Alpca API key and secret key, then insantiate an API object
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECURITY_KEY")

tradeapi = REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [4]:
def get_ticker(ticker, start, end, tradeapi):
    """Executes the get_bars method from Alpaca Trade API"""
    return tradeapi.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
    
def grade_range(one_row_df):
    """Return the high and low of a single day"""
    return one_row_df[['high', 'low']].reset_index().drop(columns='timestamp').values

In [5]:
grade_range(get_ticker("SPY", "2022-07-26", "2022-07-26", tradeapi))

array([[394.06, 389.95]])

In [67]:
class Grader:
    def __init__(self,trader_id,log_address):
        self.trader_id = trader_id
        self.log_address = log_address

In [51]:
import streamlit as st
import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
import time
from dotenv import load_dotenv

from web3 import Web3
from pathlib import Path

import json
from io import StringIO

load_dotenv()


w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

def load_deployer_contract():
    contract_address = os.getenv("DEPLOYER_ADDRESS") 
    with open(Path('deployer.json')) as f:
            deployer_abi = json.load(f)

    deployer_contract = w3.eth.contract(
        address=contract_address,
        abi=deployer_abi
        )
    return deployer_contract

def load_contract():
    contract_address = os.getenv("LOGGER_CONTRACT_ADDRESS") #could make this variable
    with open(Path('logger.json')) as f:
            logger_abi = json.load(f)

    logger_contract = w3.eth.contract(
    address=contract_address,
    abi=logger_abi
        )
    return logger_contract

def handle_new_log(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)
    

def handle_new_trade(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)
    


In [34]:
deployer = load_deployer_contract()


In [52]:
new_log_filter = deployer.events.logCreated.createFilter(fromBlock="latest")

while True:
    new_log_event = handle_new_log(new_log_filter.get_new_entries())

    print((new_log_event))
    print("--------------------------")
    print(type(new_log_event))
    time.sleep(10)

[]
--------------------------
<class 'list'>
[{'args': {'TraderId': 5, 'newLogAddress': '0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'}, 'event': 'logCreated', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0xe79708588ce0cd9efea795fc942a5df83b00707fd8281b1b8d2f2cb22b8a5a32', 'address': '0x65570CFF945218356882cE36F3bEE72aca300206', 'blockHash': '0xf30c42450e1614851d85d99226ebdd4c91f565409ec0e78485f7f894a835b32e', 'blockNumber': 42}]
--------------------------
<class 'list'>
[]
--------------------------
<class 'list'>
[]
--------------------------
<class 'list'>


KeyboardInterrupt: 

In [ ]:

contract = load_contract() 

event_filter = contract.events.newTrade.createFilter(fromBlock="latest")

while True:
    event = handle_new_trade(event_filter.get_new_entries())

    print((event))
    print("--------------------------")
    print(type(event))
    time.sleep(10)

[]
--------------------------
<class 'list'>


In [32]:
event

{'args': {'inputTraderAddress': '0x902A693035257b8f1e999E4a2414c450F5cB9E03',
  'TradeId': 6,
  'inputOpen': True,
  'inputSymbol': 'MSFT',
  'inputSize': 100,
  'inputFractional_shares': 0,
  'inputEntryPrice': 260,
  'inputExitPrice': 0,
  'inputExpirationTimeStamp': 0,
  'inputStrike': 0,
  'inputIsCall': True},
 'event': 'newTrade',
 'logIndex': 0,
 'transactionIndex': 0,
 'transactionHash': '0x4495bd80668a1894602a25eb47f6552d466de87881e8cbf1524ed930bd307287',
 'address': '0xD6e7211B6199C64C7fd3578761dCC4b14B2Ec12d',
 'blockHash': '0xb21554d0825169cc5cb963ef3bc709313f80a55e89ca7db53383618a51a39e67',
 'blockNumber': 35}

In [53]:
newLog = [{'args': {'TraderId': 5, 'newLogAddress': '0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'}, 'event': 'logCreated', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0xe79708588ce0cd9efea795fc942a5df83b00707fd8281b1b8d2f2cb22b8a5a32', 'address': '0x65570CFF945218356882cE36F3bEE72aca300206', 'blockHash': '0xf30c42450e1614851d85d99226ebdd4c91f565409ec0e78485f7f894a835b32e', 'blockNumber': 42}]

In [60]:
new_log  = newLog[0]['args']
new_log

{'TraderId': 5, 'newLogAddress': '0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'}

In [63]:
trader_dict = {}
trader_dict[new_log['TraderId']]=Grader(new_log['TraderId'],new_log['newLogAddress'])

'0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'